In [ ]:
!pip install -U fake_useragent
from requests import session
import time
import random
import json
from fake_useragent import UserAgent
from lxml import html

In [ ]:
##以Dcard API V2 爬取。存成json

def search_article(corp = '饗賓集團', brand = '饗饗'):
    
    ua = UserAgent(use_cache_server=False)
    user_agent = ua.random
    headers = {'user-agent': user_agent}
    s = session()
    
    api = 'https://www.dcard.tw/service/api/v2'
    url = api + '/search?forum=food&query='+ brand

    res = s.get(url, headers = headers)
    html = res.text
    rejs = json.loads(html)

    final_art = []
    #存取文章
    for i in rejs:
        a = {}
        a['Corp'] = corp
        a['Brand'] = brand
        a['Platform'] = "Dcard"
        a['Branch'] = ""
        a['Username'] = i['memberId']
        a['ReviewTime'] = i['createdAt'][0:10]
        a['Title'] = i['title']
        a['ReviewContent'] = i['excerpt']
        a['ReviewStar'] = ""
        a['commentCount'] = i['commentCount']
        final_art.append(a)
        
        #爬回文資訊
        url2 = api+ '/posts/'+ str(i['id']) + '/comments'
        res2 = s.get(url2, headers = headers)
        html2 = res2.text
        CmntsData = json.loads(html2)
        
        comment_content = []
        print('正在爬這篇文章', i['title'])
        time.sleep(random.randint(1,5))
        for j in CmntsData:
            try:
                b = {}
                b['Corp'] = corp
                b['Brand'] = brand
                b['Platform'] = "Dcard"
                b['Branch'] = ""
                b['Username'] = j['id']
                b['ReviewTime'] = j['createdAt'][0:10]
                b['Title'] = i['title']
                b['ReviewContent'] = j['content']
                b['ReviewStar'] = ""
                b['commentCount'] = ""
                comment_content.append(b)
                print('正爬到第', j['floor'], '樓')
                time.sleep(random.randint(1,3))
            except KeyError:
                print('留言被刪除')
                continue
   
        for i in comment_content:
            final_art.append(i)
            
    return final_art


def output(filename, data):
    try:
        with open(filename +".json", 'wb+') as f:
            f.write(json.dumps(data, indent=1, ensure_ascii=False).encode('utf-8'))
            print('爬取完成', filename + '.json', '輸出成功')
    except Exception as err:
        print(filename +'.json', '輸出失敗')
        print('error message:', err)

In [ ]:
#建立字典與陣列以便做迴圈

dict1= {'饗賓集團':['饗饗','旭集','饗泰多','饗食天堂','果然匯','小福利火鍋','開飯川食堂','真珠'],
        '王品集團':['12mini','丰禾','王品','石二鍋','肉次方','西提','尬鍋','享鴨','和牛涮','青花驕','品田牧場','原燒','夏慕尼','莆田','陶板屋','最肉燒肉','聚北海道鍋物','藝奇','hot7','the wang']}
corp1 = '饗賓集團'
corp2 = '王品集團'

In [ ]:
#饗賓迴圈執行完 接著執行王品迴圈
for a in range(0,len(dict1[corp1])):
    data = search_article(corp = corp1, brand = dict1[corp1][a])
    output(dict1[corp1][a], data)

#    
for a in range(0,len(dict1[corp2])):
    data = search_article(corp = corp2, brand = dict1[corp2][a])
    output(dict1[corp2][a], data)

In [ ]:
## json to csv

import json
import pandas as pd

for b in range(0,len(dict1[corp1])):
    with open(dict1[corp1][b]+'.json', 'r', encoding = "utf-8") as f:
        a = json.loads(f.read())

        df = pd.json_normalize(a)
        df.to_csv('Dcard_'+dict1[corp1][b]+'.csv', encoding='utf-8-sig', index=False)

for b in range(0,len(dict1[corp2])):
    with open(dict1[corp2][b]+'.json', encoding = "utf-8") as f:
        a = json.loads(f.read())

        df = pd.json_normalize(a)
        df.to_csv('Dcard_'+dict1[corp2][b]+'.csv', encoding='utf-8-sig', index=False)


In [ ]:
##合併所有 Dard_ 開頭的csv檔案。存成concatted7_final.csv，不要index，並且以utf-8-sig編碼

import pandas as pd
from glob import glob
 
files = glob('Dcard_*.csv')
df2 = pd.concat((pd.read_csv(file) for file in files), ignore_index=True)
df2.to_csv( 'concatted7_final.csv', index=False, encoding="utf-8-sig" )

In [8]:
#有留言換行問題的通通丟進來重新存一次，再用Tableau看才ＯＫ

import pandas as pd
from glob import glob
 
alist=['饗饗','旭集','饗泰多','饗食天堂','果然匯','小福利火鍋','開飯川食堂','真珠','12mini','丰禾','王品','石二鍋','肉次方','西提','尬鍋','享鴨','和牛涮','青花驕','品田牧場','原燒','夏慕尼','莆田','陶板屋','最肉燒肉','聚北海道鍋物','藝奇','hot7','thewang']
    
for brandname1 in alist:
    files = glob('Dcard_'+str(brandname1)+'.csv')
    df1 = pd.concat((pd.read_csv(file) for file in files), ignore_index=True)
    df1.to_csv( 'dcard_'+str(brandname1)+'_new.csv', index=False, encoding="utf-8-sig" )
    print ('dcard_'+str(brandname1)+'_new.csv存取完成')

dcard_饗饗_new.csv存取完成
dcard_旭集_new.csv存取完成
dcard_饗泰多_new.csv存取完成
dcard_饗食天堂_new.csv存取完成
dcard_果然匯_new.csv存取完成
dcard_小福利火鍋_new.csv存取完成
dcard_開飯川食堂_new.csv存取完成
dcard_真珠_new.csv存取完成
dcard_12mini_new.csv存取完成
dcard_丰禾_new.csv存取完成
dcard_王品_new.csv存取完成
dcard_石二鍋_new.csv存取完成
dcard_肉次方_new.csv存取完成
dcard_西提_new.csv存取完成
dcard_尬鍋_new.csv存取完成
dcard_享鴨_new.csv存取完成
dcard_和牛涮_new.csv存取完成
dcard_青花驕_new.csv存取完成
dcard_品田牧場_new.csv存取完成
dcard_原燒_new.csv存取完成
dcard_夏慕尼_new.csv存取完成
dcard_莆田_new.csv存取完成
dcard_陶板屋_new.csv存取完成
dcard_最肉燒肉_new.csv存取完成
dcard_聚北海道鍋物_new.csv存取完成
dcard_藝奇_new.csv存取完成
dcard_hot7_new.csv存取完成


ValueError: No objects to concatenate

In [35]:
import pandas as pd
from glob import glob
 
files = glob('concatted6_*.csv')
df2 = pd.concat((pd.read_csv(file) for file in files), ignore_index=True)
df2.to_csv( 'concatted7_final.csv', index=False, encoding="utf-8-sig" )

/var/folders/42/7l02yz4j6vggnl38lsbgsv4r0000gn/T/ipykernel_31565/1026784621.py:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.concat((pd.read_csv(file) for file in files), ignore_index=True)


In [36]:
df2

,Corp,Brand,Platform,Branch,Username,ReviewTime,Title,ReviewContent,ReviewStar,CommentCount
0,王品集團,王品牛排,GoogleReviews,台北羅斯福店,april Chou,2022/5/13,NaN,NaN,5.0,NaN
1,王品集團,王品牛排,GoogleReviews,台北羅斯福店,Ricky Lee (朣言童語),2022/5/13,NaN,大概十年吃一次…..,4.0,NaN
2,王品集團,王品牛排,GoogleReviews,台北羅斯福店,LiLi,2022/5/13,NaN,車程五分鐘，外帶所有餐點都冷掉了，非常影響食慾cp值低,1.0,NaN
3,王品集團,王品牛排,GoogleReviews,台北羅斯福店,克羅諾,2022/5/6,NaN,NaN,4.0,NaN
4,王品集團,王品牛排,GoogleReviews,台北羅斯福店,AJ ahn,2022/5/6,NaN,(由 Google 提供翻譯) 好吃??\n\n(原始評論)\n好吃??,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...
261812,饗賓集團,果然匯,Ptt,NaN,patric1224,2021/11/1,[問卦] 果然匯要吃什麼比較內行？,素食吃到飽根本就是一般素食拿掉肉，,NaN,NaN
261813,饗賓集團,果然匯,Ptt,NaN,patric1224,2021/11/1,[問卦] 果然匯要吃什麼比較內行？,並沒有比較好吃或特別。,NaN,NaN
261814,饗賓集團,果然匯,Ptt,NaN,patric1224,2021/11/1,[問卦] 果然匯要吃什麼比較內行？,"更正,一般吃到飽拿掉肉。",NaN,NaN
261815,饗賓集團,果然匯,Ptt,NaN,GossipCandy,2021/11/1,[問卦] 果然匯要吃什麼比較內行？,松露奶油燉飯好ㄘ,NaN,NaN


In [ ]:
##martin提供

import random
from bs4 import BeautifulSoup as bs
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager  # 自動更新webdriver pip install webdriver-manager
import time
from fake_useragent import UserAgent

search = "王品"
ua = UserAgent()
options = Options()
options.add_argument("user-agent=" + ua.chrome)
# options.add_argument("--headless")
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.get("https://www.dcard.tw/search?forum=food&query={}".format(search))

temp_height = 0
link_list = []
while True:
    driver.execute_script("window.scrollBy(0,1000)")
    time.sleep(random.randint(5, 15))
    check_height = driver.execute_script("return document.documentElement.scrollTop || window.pageYOffset || document.body.scrollTop;")
    if check_height == temp_height:
        break
    temp_height = check_height
driver.quit()

soup = bs(driver.page_source, 'lxml')
url = soup.find_all('a', class_="sc-b205d8ae-3 iOQsOu")
link_list.append("https://www.dcard.tw" + url.get("href"))

dic = {}
dic["link"] = link_list
ls = []
ls.append(dic)
with open("link.json","r",encoding="utf-8") as file:
    json.dump(ls, file, ensure_ascii=False)